# 6.0002 Lecture 6: Monte Carlo Simulation

**Speaker:** Prof. John Guttag

## A Little History
- Stanislav Ulam, recovering from an illness, was playing a lot of solitaire
- Tried to figure out probability of winning, and failed
    - combinatorics were too complicated
- Thought about playing lots of hands and counting the number of wins, but decided it would take years
- Asked Von Neumann if he could build a program to simulate many hands on ENIAC

## Monte Carlo Simulation
- a method of estimating the value of an unknown quantity using the principles of inferential statistics
- **inferential statistics**
    - *population*: a set of examples
    - *sample*: a proper subset of a population
    - key fact: a **random sample** (sample chosen at random from population) tends to exhibit the same properties as the population from which it is drawn
- exactly what we did with random walks

## An example
- given a single coin, estimate the fraction of heads you would get if you flipped the coin an infinite number of times
- consider one flip
    - how confident would you be about answering 1.0?

## Flipping a coin twice
- do you think that the next flip will come up heads?

## Flipping a coin 100 times
- imagine all 100 flips came up as heads
    - now do you think that the next flip will come up heads?
- what if 52 of them came up heads?
    - do you think that the probability of the next flip coming up heads is 52/100?
    - given the data, it's your best estimate
    - but confidence should be low

## Why the difference in confidence?
- confidence in our estimate depends upon two things:
    - **size** of sample (e.g. 100 vs 2)
    - **variance** of sample (e.g. all heads versus 52 heads)
- as the variance grows, we need larger samples to have the same degree of confidence

## Roulette
- no need to simulate, since answers obvious
- allows us to compare simulation results to actual probabilities

## Class definition

In [6]:
import random

class FairRoulette():
    def __init__(self):
        self.pockets = []
        for i in range(1, 37):
            self.pockets.append(i)
        self.ball = None
        self.pocketOdds = len(self.pockets) - 1
    def spin(self):
        self.ball = random.choice(self.pockets)
    def betPocket(self, pocket, amt):
        if str(pocket) == str(self.ball):
            return amt*self.pocketOdds
        else:
            return -amt
    def __str__(self):
        return 'Fair Roulette'

## Monte Carlo simulation

In [9]:
def playRoulette(game, numSpins, pocket, bet, toPrint=False):
    totPocket = 0
    for i in range(numSpins):
        game.spin()
        totPocket += game.betPocket(pocket, bet)
    if toPrint:
        print(numSpins, 'spins of', game)
        print('Expected return betting', pocket, '=',\
              str(100*totPocket/numSpins) + '%\n')
    return (totPocket/numSpins)

In [10]:
game = FairRoulette()
for numSpins in (100, 1000000):
    for i in range(3):
        playRoulette(game, numSpins, 2, 1, True)

100 spins of Fair Roulette
Expected return betting 2 = -64.0%

100 spins of Fair Roulette
Expected return betting 2 = 8.0%

100 spins of Fair Roulette
Expected return betting 2 = 116.0%

1000000 spins of Fair Roulette
Expected return betting 2 = -0.4924%

1000000 spins of Fair Roulette
Expected return betting 2 = -0.3268%

1000000 spins of Fair Roulette
Expected return betting 2 = 0.692%



- for the simulations with 1,000,000 spins, we see that there is much less variance among the results

## Law of Large Numbers
- in repeated independent tests with the same actual probability $p$ of a particular outcome in each test, the chance that the fraction of times that outcome occurs differs from $p$ converges to 0 as the number of trials goes to infinity
    - does this imply that if deviations from expected behavior occur, these deviations are likely to be **evened out** by opposite deviations in the future?

## Gambler's Fallacy
- "On August 18, 1913, at the casino in Monte Carlo, black came up a rrecord twenty-six times in succession [in roulette]. ...[There] was a near-panicky rush to bet on red, beginning about the time black had come up a phenomenal fifteen times." -- Huff and Geis, *How to Take a Chance*
- probability of 26 consecutive reds:
    - 1/67,108,865
- probability of consecutive reds when previous 25 rolls were red:
    - 1/2
    - because independent events

## Regression to the Mean
- following an extreme random event, the next random event is likely to be less extreme
- if you spin a fair roulette wheel 10 times and get 100% reds, that is an extreme event (probability 1/1024)
    - gambler's fallacy: if we spin it another 10 times, it needs to even out, i.e. we should get more blacks than usual to make up for excess reds
    - regression to mean is different
- it is likely that in the next 10 spins, you will get fewer than 10 reds
    - but the expected number is only 5
- so if you look at the average of the 20 spins, it will be closer to the expected mean of 50% reds than to the 100% of the first 10 spins

## Casinos not in the business of being fair
- Casinos in Europe often sneak in one green slot into the red and black slots of being roulette
    - index slots from zero rather than from 1
    - so you don't get a full payoff
- some casinos in the US even have two green slots
    - zero, and double zero

## Two Subclasses of Roulette

In [12]:
# pockets now include 0
class EuRoulette(FairRoulette):
    def __init__(self):
        FairRoulette.__init__(self)
        self.pockets.append('0')
    def __str__(self):
        return 'European Roulette'

In [13]:
# pockets now include 0 and 00
class AmRoulette(EuRoulette):
    def __init__(self):
        EuRoulette.__init__(self)
        self.pockets.append('00')
    def __str__(self):
        return 'American Roulette'

## Comparing the games

In [16]:
# helper code
def findPocketReturn(game, numTrials, trialSize, toPrint):
    pocketReturns = []
    for t in range(numTrials):
        trialVals = playRoulette(game, trialSize, 2, 1, toPrint)
        pocketReturns.append(trialVals)
    return pocketReturns

In [17]:
# helper code
def getMeanAndStd(X):
    mean = sum(X)/float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    std = (tot/len(X))**0.5
    return mean, std

In [18]:
# simulation
random.seed(0)
numTrials = 20
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette)
for G in games:
    resultDict[G().__str__()] = []
for numSpins in (1000, 10000, 100000, 1000000):
    print('\nSimulate', numTrials, 'trials of',
          numSpins, 'spins each')
    for G in games:
        pocketReturns = findPocketReturn(G(), numTrials,
                                         numSpins, False)
        expReturn = 100*sum(pocketReturns)/len(pocketReturns)
        print('Exp. return for', G(), '=',
             str(round(expReturn, 4)) + '%')


Simulate 20 trials of 1000 spins each
Exp. return for Fair Roulette = 6.56%
Exp. return for European Roulette = -2.26%
Exp. return for American Roulette = -8.92%

Simulate 20 trials of 10000 spins each
Exp. return for Fair Roulette = -1.234%
Exp. return for European Roulette = -4.168%
Exp. return for American Roulette = -5.752%

Simulate 20 trials of 100000 spins each
Exp. return for Fair Roulette = 0.8144%
Exp. return for European Roulette = -2.6506%
Exp. return for American Roulette = -5.113%

Simulate 20 trials of 1000000 spins each
Exp. return for Fair Roulette = -0.0723%
Exp. return for European Roulette = -2.7329%
Exp. return for American Roulette = -5.212%


- clealry, fair roulette seems to be a better bet in all cases

## Sampling space of possible outcomes
- never possible to guarantee perfect accuracy through sampling
- not to say that an estimate is not precisely correct
- key question:
    - how many samples do we need to look at before we can have justified confidence on our answer?
- depends upon variability in underlying distribution

## Quantifying variation in data
- Let $X$ be a list of data items (observations)
- **variance**: $$Var(X) = \frac{\sum_{x\in X}(x-\mu)^2}{|X|}$$
- **standard deviation**: $$\sigma(X)=\sqrt{\frac{1}{|X|}\sum_{x\in X}(x-\mu)^2}$$
- standard deviation $\sigma(X)$ simply the square root of the variance
- outliers can have a big effect
- standard deviation should always be considered relative to the mean $\mu$ of $X$

## For those who prefer code

In [19]:
def getMeanAndStd(X):
    """Assumes X is a list of numbers
        Returns the mean and standard 
        deviation of X"""
    mean = sum(X)/float(len(X))
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    std = (tot/len(X))**0.5
    return mean, std

## Confidence Levels and Intervals
- instead of estimating an unknown parameter by a single value (e.g. the mean of a set of trials), a confidence interval provides a range that is likely to contain the unknown value and a confidence that the unknown value lies within that range
- "The return on betting a pocket 10k times in European roulette is -3.3%. The margin of error is +/- 3.5% with a 95% level of confidence."
    - what does this mean?
- If I were to conduct an infinite number of trials of 10k bets each, 
     - my expected average return would be -3.3%
     - my return would be roughly within [-6.8%,+0.2%] 95% of the time

## Empirical Rule
- under some assumptions discussed later
    - ~68% of data within 1 standard deviation of mean
    - ~95% of data within 1.96 standard deviations of mean
    - ~99.7% of data within 3 standard deviations of mean

## Apply Empirical Rule

In [21]:
resultDict = {}
games = (FairRoulette, EuRoulette, AmRoulette)

for G in games:
    resultDict[G().__str__()] = []
for numSpins in (100, 1000, 10000):
    print('\nSimulate betting a pocket for', numTrials, 
          'trials of', numSpins, 'spins each')
    for G in games:
        pocketReturns = findPocketReturn(G(), 20,
                                         numSpins, False)
        mean, std = getMeanAndStd(pocketReturns)
        resultDict[G().__str__()].append((numSpins,
                                          100*mean,
                                          100*std))
        # print the confidence intervals
        print('Exp. return for', G(), '=',
              str(round(100*mean, 3))
              + '%,', '+/- ' + str(round(100*1.96*std,3))
              + '% with 95% confidence')


Simulate betting a pocket for 20 trials of 100 spins each
Exp. return for Fair Roulette = -2.8%, +/- 116.156% with 95% confidence
Exp. return for European Roulette = -28.0%, +/- 77.295% with 95% confidence
Exp. return for American Roulette = -11.8%, +/- 147.122% with 95% confidence

Simulate betting a pocket for 20 trials of 1000 spins each
Exp. return for Fair Roulette = -4.24%, +/- 34.234% with 95% confidence
Exp. return for European Roulette = -2.8%, +/- 28.136% with 95% confidence
Exp. return for American Roulette = 4.22%, +/- 35.03% with 95% confidence

Simulate betting a pocket for 20 trials of 10000 spins each
Exp. return for Fair Roulette = 1.358%, +/- 11.674% with 95% confidence
Exp. return for European Roulette = -1.414%, +/- 14.178% with 95% confidence
Exp. return for American Roulette = -6.742%, +/- 6.269% with 95% confidence


## Assumptions underlying Empirical Rule
- the mean estimation error is zero
    - "just as likely to guess high as to guess low"
    - different in a lab experiment, where technique may cause a bias in results towards one direction
    - so we are assuming no bias in our errors
- the distribution of the errors in the estimate is normal (Gaussian)

## Defining Distributions
- use a probability distribution
- captures notion of relative frequency with which a random variable takes on certain values
    - discrete random variables drawn from a finite set of values
    - continuous random variables drawn from reals between two numbers (i.e. infinite set of values)
- for discrete variable, simply list the probability of each value
    - must add up to 1
- continuous case trickier; can't just enumerate probability for each of an (uncountably) infinite set of values

## PDF's
- distributions defined by *probability density functions* (PDFs)
- probability of a random variable lying between two values
- defines a curve where the values on the x-axis lie between minimum and maximum value of the variable
- area under curve between two points, is probability of example falling within that range

## Normal Distributions
- pdf for a Gaussian: $$f(x) = \frac{1}{\sigma \sqrt{2\pi}}e^{\frac{(x-\mu)^2}{2\sigma^2}}$$
- definition of the number $e$: $$e = \sum_{n=0}^{\infty}\frac{1}{n!}$$
- the probability that random variable $X$ lies within an interval $[a,b]\subset \mathbb{R}$:
$$\mathbb{P}(a \leq X \leq b) = \int_{a}^b f(x) \textrm{d}x$$
- ~68% of data within one standard deviation of mean:
$$0.68 = \int_{-1}^{1}f(x)\textrm{d}x$$
- ~95% of data within 1.96 standard deviations of mean:
$$0.95 = \int_{-1.96}^{1.96} f(x) \textrm{d}x$$